In [1]:
import os
import sys
import tempfile
from pathlib import Path

# Set the path to the directory containing the project
sys.path.append('D:\\Internship\\recsys\\db_handlers')
sys.path.append('D:\\Internship\\recsys\\movie_recommendation_system\\src')

In [2]:
from db_handlers.trained_models_minio_storage_db_handler import (
    create_initial_model_bucket,
    create_offline_updated_models_bucket,
    create_online_updated_user_models_bucket,
    drop_buckets,
    reset_buckets,
    upload_initial_model,
    upload_offline_updated_model,
    upload_online_user_model,
    download_initial_model,
    download_offline_updated_model,
    download_last_offline_updated_model,
    download_online_user_model
)

[LOG] Detected local environment, with Docker: [False]


In [3]:
# Create buckets
create_initial_model_bucket()
create_offline_updated_models_bucket()
create_online_updated_user_models_bucket()

[LOG] Bucket 'initial-model' already exists.
[LOG] Bucket 'initial-model' created successfully in [local] at URL: 192.168.1.29:9500
[LOG] Bucket 'offline-updated-models' created.
[LOG] Bucket 'offline-updated-models' created successfully in [local] at URL: 192.168.1.29:9500
[LOG] Bucket 'online-updated-user-models' created.
[LOG] Bucket 'online-updated-user-models' created successfully in [local] at URL: 192.168.1.29:9500


In [4]:
trained_models_dir = 'D:\\Internship\\trained_models'
sample_model_filepath = os.path.join(trained_models_dir, 'GraphSAGE_based_model.pth')
download_folder = os.path.join(trained_models_dir, "downloaded")

# Test storing sample model
upload_initial_model(model_filepath=sample_model_filepath)
upload_offline_updated_model(model_filepath=sample_model_filepath, date="2023-10-10", drop_old=False)
upload_offline_updated_model(model_filepath=sample_model_filepath, date="2023-10-11", drop_old=True)
upload_online_user_model(model_filepath=sample_model_filepath, user_id="sample_user")

[LOG] Initial pre-trained model 'init_model.pth' uploaded.
[LOG] Offline updated model '2023-10-10_offline_model.pth' uploaded.
[LOG] Offline updated model '2023-10-11_offline_model.pth' uploaded.
[LOG] Old model '2023-10-10_offline_model.pth' removed.
[LOG] Online user model for user 'sample_user' uploaded.


In [5]:
# Test retrieving sample model
download_initial_model(download_path=download_folder)
print(f"Downloaded sample model to {download_folder}\n")

try:
    download_offline_updated_model(download_path=download_folder, date="2023-10-10")
    print(f"Downloaded offline updated sample model to {download_folder}\n")
except Exception as e:
    print(f"[ERROR] Failed to download initial model: {e}\n")

download_offline_updated_model(download_path=download_folder, date="2023-10-11")
print(f"Downloaded offline updated sample model to {download_folder}\n")

download_last_offline_updated_model(download_path=download_folder)
print(f"Downloaded last offline updated sample model to {download_folder}\n")

download_online_user_model(download_path=download_folder, user_id="sample_user")
print(f"Downloaded online user sample model to {download_folder}")

[LOG] Initial model 'init_model.pth' downloaded as 'init_model.pth'.
Downloaded sample model to D:\Internship\trained_models\downloaded

[ERROR] Failed to download initial model: S3 operation failed; code: NoSuchKey, message: Object does not exist, resource: /offline-updated-models/2023-10-10_offline_model.pth, request_id: 1834C2150C80836B, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: offline-updated-models, object_name: 2023-10-10_offline_model.pth

[LOG] Offline updated model '2023-10-11_offline_model.pth' downloaded as '2023-10-11_offline_model.pth'.
Downloaded offline updated sample model to D:\Internship\trained_models\downloaded

[LOG] Latest offline updated model '2023-10-11_offline_model.pth' downloaded as '2023-10-11_offline_model.pth'.
Downloaded last offline updated sample model to D:\Internship\trained_models\downloaded

[LOG] Online user model for user 'sample_user' downloaded as 'sample_user_online_model.pth'.
Downloaded online u

In [6]:
from movie_recommender.models.gnn_train_eval_pred import GNNModelHandler

# Test whether the downloaded model was serialized correctly and works as expected
sample_model = GNNModelHandler.load_pretrained_model(sample_model_filepath)
gdh = sample_model._gdh

In [7]:
# Sample prediction
user_id = 672
movie_id = 1891
pred_rating, gt_rating = sample_model.predict(user_id, movie_id)

# Extract movie titles as a list
movie_titles = gdh.movies_df.loc[gdh.movies_df['id'] == movie_id, 'title'].tolist()

# Check if any titles were found
if movie_titles:
    movie_title = movie_titles[0]   # Display the first title
    print(f"Movie: {movie_title}, Pred rating: {pred_rating:.4f}, Ground truth rating: {gt_rating:.4f}")
else:
    print(f"No movie found with id {movie_id}")

Movie: The Empire Strikes Back, Pred rating: 2.6529, Ground truth rating: 2.5000


In [8]:
# Reset buckets
reset_buckets()

[LOG] Bucket 'initial-model' has been emptied.
[LOG] Bucket 'offline-updated-models' has been emptied.
[LOG] Bucket 'online-updated-user-models' has been emptied.


In [9]:
# Drop buckets
drop_buckets()

[LOG] Bucket 'initial-model' and its contents deleted.
[LOG] Bucket 'offline-updated-models' and its contents deleted.
[LOG] Bucket 'online-updated-user-models' and its contents deleted.
